In [6]:

import torch
import torch.nn as nn
import torch.nn.functional as functional
from tifffile import imread as tiff_read
from tifffile import imwrite as tiff_write
from model_define import Algin
from torch.utils.data import DataLoader
import torchfields
from torchinfo import summary
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

if __name__ == "__main__":

    print('trainning align model')
    gd = torch.load('dataset1/train_gd.pt')
    gd/=2455 #maxinum of grey value
    wrapped = torch.load('dataset1/train_wrapped.pt')
    wrapped /=2455
    print('data loaded')
    dataset=torch.utils.data.TensorDataset(wrapped,gd)
    train_dataloader = torch.utils.data.DataLoader(dataset,batch_size=2,num_workers=1)
    model = Algin(1)
    model = model
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    result = []
    for i in trange(1):
        for train_features, train_labels in train_dataloader:
            train_features = train_features
            train_labels = train_labels
            predict_field = model(train_labels,train_features).field()
            x_predict = predict_field(train_features)
            loss = loss_fn(x_predict,train_labels)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        current_result = loss.sum().item()
        print(current_result)

In [3]:
s = 'shit'
f'{s}fuck'

'shitfuck'

In [8]:

print('model_define as main')
#test FeatureExtraction block
f = FeatureExtraction(1)
t = torch.rand(1,1,64,64)
print(t)
t = f(t)
for fe in t:
    print(fe.shape)
g = Algin(1)

model_define as main
tensor([[[[0.0268, 0.2877, 0.0921,  ..., 0.1785, 0.3015, 0.6776],
          [0.3145, 0.3739, 0.9251,  ..., 0.4944, 0.2459, 0.9978],
          [0.3593, 0.7910, 0.8134,  ..., 0.5160, 0.6391, 0.4028],
          ...,
          [0.4550, 0.3278, 0.4619,  ..., 0.0608, 0.3335, 0.2610],
          [0.7901, 0.5826, 0.8488,  ..., 0.8658, 0.2250, 0.6202],
          [0.8745, 0.9346, 0.2246,  ..., 0.3479, 0.4916, 0.3761]]]])
torch.Size([1, 16, 64, 64])
torch.Size([1, 16, 64, 64])
torch.Size([1, 16, 64, 64])


In [9]:
t2 = torch.rand(1,1,64,64)

In [14]:
for i in range(10-1,-1,-1):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [11]:
u = g(t2,t2)

IndexError: index 3 is out of range